In [302]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()
if scope._is_husky:
    scope.adc.samples = 80
else:
    scope.adc.samples = 129
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "disabled"

#TARGET_PLATFORM = 'CW305_100t'
TARGET_PLATFORM = 'CW305_35t'
# TARGET_PLATFORM = 'CW312T_A35'
#TARGET_PLATFORM = 'CW312T_ICE40'

if TARGET_PLATFORM in ['CW312T_A35', 'CW312T_ICE40']:
    scope.io.hs2 = 'clkgen'
    fpga_id = None # not needed
    if TARGET_PLATFORM == 'CW312T_A35':
        platform = 'ss2_a35'
        scope.gain.db = 45 # this is a good setting for the inductive shunt; if using another, adjust as needed
    else:
        platform = 'ss2_ice40'
        scope.gain.db = 15
else:
    scope.gain.db = 25
    scope.io.hs2 = "disabled"
    platform = 'cw305'
    if TARGET_PLATFORM == 'CW305_100t':
        fpga_id = '100t'
    elif TARGET_PLATFORM == 'CW305_35t':
        fpga_id = '35t'

# On the CW305, setting force=False only programs the FPGA if it is currently unprogrammed, whereas force=True programs the FPGA regardless.
# This option isn't available on the CW312T_A35 or CW312T_ICE40.
# target = cw.target(scope, cw.targets.CW305, force=True, fpga_id=fpga_id, platform=platform)

target = cw.target(None, cw.targets.CW305, force=True, bsfile="/home/boochoo/hqc/dummy-insertion-sparse-polymult/CW305/poly_mult/vivado_polymult_vulnerable_cw305/vivado_polymult_vulnerable_cw305.runs/impl_1/cw305_top.bit")


scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 566395                    to 21121375                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 29538459                 
scope.clock.adc_rate                     changed from 0.0                       to 29538459.0               
scope.clock.freq_ct

(ChipWhisperer Target WARNING|File CW305.py:591) Using default Verilog defines (/home/boochoo/chipwhisperer/software/chipwhisperer/hardware/firmware/cw305/cw305_aes_defines.v); if this is not what you want, provide them via the defines_files argument


In [303]:
if TARGET_PLATFORM in ['CW305_100t', 'CW305_35t']:
    target.vccint_set(1.0)
    # we only need PLL1:
    target.pll.pll_enable_set(True)
    target.pll.pll_outenable_set(False, 0)
    target.pll.pll_outenable_set(True, 1)
    target.pll.pll_outenable_set(False, 2)

    # run at 10 MHz:
    target.pll.pll_outfreq_set(10E6, 1)

    # 1ms is plenty of idling time
    target.clkusbautooff = True
    target.clksleeptime = 1

if TARGET_PLATFORM in ['CW305_100t', 'CW305_35t']:
    if scope._is_husky:
        scope.clock.clkgen_freq = 10e6
        scope.clock.clkgen_src = 'extclk'
        scope.clock.adc_mul = 4
        # if the target PLL frequency is changed, the above must also be changed accordingly
    else:
        scope.clock.adc_src = "extclk_x4"
        
if TARGET_PLATFORM in ['CW312T_A35', 'CW312T_ICE40']:
    scope.clock.clkgen_freq = 7.37e6
    scope.io.hs2 = 'clkgen'
    if scope._is_husky:
        scope.clock.clkgen_src = 'system'
        scope.clock.adc_mul = 4
        scope.clock.reset_dcms()
    else:
        scope.clock.adc_src = "clkgen_x4"
    import time
    time.sleep(0.1)
    target._ss2_test_echo()

import time
for i in range(5):
    scope.clock.reset_adc()
    time.sleep(1)
    if scope.clock.adc_locked:
        break 
assert (scope.clock.adc_locked), "ADC failed to lock"

In [304]:
project_file = "projects/polymult_vulnerable.cwp"
project = cw.create_project(project_file, overwrite=True)

In [305]:
def read_y_128_mem_as_int_list(filename):
    """ 
    y_128.mem 파일을 읽고 각 줄을 2진수에서 정수로 변환하여 리스트로 반환 
    """
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    # 2진수를 정수(int)로 변환하여 리스트에 저장
    int_list = [int(line.strip(), 2) for line in lines]
    
    return int_list


int_values = read_y_128_mem_as_int_list("/home/boochoo/hqc/dummy-insertion-sparse-polymult/CW305/poly_mult/jupyter/y_128.mem")
normal_values = read_y_128_mem_as_int_list("/home/boochoo/hqc/dummy-insertion-sparse-polymult/CW305/poly_mult/jupyter/h_for_y_32.mem")
# print(normal_values[0])
print(normal_values[551])

309011921


In [306]:
def number_to_bytearray(num, high_bits=0, size=16):
    """ 
    숫자를 지정된 바이트 크기로 변환하는 함수.
    high_bits (0~3): 최상위 2비트 (128, 127 비트)를 설정할 값
    """
    if not (0 <= high_bits <= 3):
        raise ValueError("high_bits는 0~3 사이의 값이어야 합니다.")

    # 126비트 값만 유지하고, 최상위 2비트는 high_bits로 변경
    num = (num & ((1 << 126) - 1)) | (high_bits << 126)

    return num.to_bytes(size, byteorder='little')


In [307]:
for i in range(len(int_values) + len(normal_values)):
    target.fpga_write(target.REG_CRYPT_KEY, number_to_bytearray(i,0))
    if i < len(int_values):
        target.fpga_write(target.REG_CRYPT_TEXTIN, int_values[i].to_bytes(16, byteorder='little'))
    else:
        target.fpga_write(target.REG_CRYPT_TEXTIN, normal_values[i-len(int_values)].to_bytes(16, byteorder='little'))
    target.go()


In [313]:
def read_key(index):
    target.fpga_write(target.REG_CRYPT_TEXTIN, bytearray(16))
    target.fpga_write(target.REG_CRYPT_KEY, number_to_bytearray(index, 0))
    target.go()
    result_byte = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)
    return int.from_bytes(result_byte, byteorder='little')


print(read_key(0))
print(target.fpga_read(target.REG_CRYPT_TEXTIN, 16))

148
bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


In [ ]:
target.fpga_write(target.REG_CRYPT_TEXTIN, bytearray([0xFF] * 16))
target.go()
print(target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16))


10111000010000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [134]:
for i in range(1):
    print("-----")
    target.fpga_write(target.REG_CRYPT_TEXTIN, data_32bit[i])
    print(''.join(format(byte, '08b') for byte in data_32bit[i]))

    # a = target.fpga_read(target.REG_CRYPT_TEXTIN, 16)
    # print(''.join(format(byte, '08b') for byte in a))

    target.fpga_write(target.REG_CRYPT_KEY, number_to_bytearray(4 * i, 1))
    print(''.join(format(byte, '08b') for byte in number_to_bytearray(4 * i, 1)))
    
    target.go()
    a = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)
    print(''.join(format(byte, '08b') for byte in a))





-----
00011010000010100001000101100000001000110011001000111000111010010011001111111010110011100001101111110110000000010101100010011100
01000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00011010000010100001000101100000010000001000101000111110100000110011110110111011001111011001000100111101010010100011110011110111


In [ ]:
for i in range(10):
    a = number_to_bytearray(i, 2)
    print(''.join(format(byte, '08b') for byte in a))
    target.fpga_write(target.REG_CRYPT_KEY, number_to_bytearray(i, 2))
    target.go()


10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [103]:
a = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)
print(''.join(format(byte, '08b') for byte in a))

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [80]:
target.fpga_write(target.REG_CRYPT_TEXTIN, bytearray([0x34]))
target.fpga_write(target.REG_CRYPT_KEY, bytearray([0x22]))

In [31]:
from tqdm.notebook import tnrange
import numpy as np
import time
from Crypto.Cipher import AES

ktp = cw.ktp.Basic()

key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

ret = cw.capture_trace(scope, target, text, key)

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08
(ChipWhisperer Scope WARNING|File __init__.py:517) Timeout happened during capture


In [33]:
print(text)
print(key)

CWbytearray(b'85 82 4f bd 8a 87 6a fe 3a bb 30 17 50 34 7f cb')
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


In [ ]:
# from tqdm.notebook import tnrange
# import numpy as np
# import time
# from Crypto.Cipher import AES

# ktp = cw.ktp.Basic()

# traces = []
# textin = []
# keys = []
# N = 5000  # Number of traces

# # initialize cipher to verify DUT result:
# key, text = ktp.next()
# cipher = AES.new(bytes(key), AES.MODE_ECB)

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
CWbytearray(b'77 a5 1c f4 6b ac 9f 37 60 2f 23 e4 7f 32 04 a8')
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
CWbytearray(b'36 c4 c0 c8 0f b2 54 a3 fc 6a b4 0c 13 50 a9 af')
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
CWbytearray(b'cf 37 4f 34 20 f7 e9 ef 2e 4c 17 f1 2a 3b 02 34')
